In [1]:
import boto3

# Working with Bedrock

Until now, I have worked with OpenAI's API. I wanted to learn to use Bedrock. I came across the the website [AWS in plain English](https://aws.plainenglish.io/). They had blogs on connecting to Bedrock. So I wanted to try it out. Here is my attempt.

## Gotchas
Bedrock was not available as a client in `langchain`. Even when I did `pip install boto3 --upgrade`, it would not work. So I had to do the following:

1. Shut down Jupyter.
2. Uninstall `boto3` and `botocore`
3. Re-install `boto3` 
4. Launch Jupyter notebook

I used 2 different blog articles for working on this code.

* [_Getting Started with Bedrock_ blog](https://medium.com/@charlesdouglas_96859/getting-started-with-aws-bedrock-33eee356af72) article helped me setup the basic authentication and issue a simple request to Claude. 
* For the RAG implementation with Bedrock, I used the blog, [Chat with your data..](https://aws.plainenglish.io/chat-with-your-data-a-simple-guide-using-amazon-bedrock-langchain-and-streamlit-2d60ea857eaf).

The document I am summarizing is a paper from Anthropic called [Consutitutional AI: Harmlessness from AI Feedback](https://arxiv.org/pdf/2212.08073.pdf). 

My ideal end-goal is to convert this to some kind of a Lambda code which can accept user input and respond back with the AI generated response. For this, I'll probably refer to the [Intelligent Document Processing](https://aws.amazon.com/blogs/machine-learning/intelligent-document-processing-with-amazon-textract-amazon-bedrock-and-langchain/) AWS blog.

## Bedrock with Boto3

The following code just tries to connect to Bedrock and list the available models. The purpose of this code is to ensure that `boto3` is able to read the credentials and Bedrock is accessible for this user.

In [29]:
# Create an Bedrock client
# although they say that boto3.client('bedrock') is possible, it would not work for me. I had to break it to 2 steps.
session = boto3.Session(profile_name='aws_sol')
bedrock = session.client('bedrock') 

In [13]:
resp = bedrock.list_foundation_models()

In [14]:
for model in resp['modelSummaries']:
    print(model['modelName'])

Titan Text Large
Titan Image Generator G1
Titan Image Generator G1
Titan Text Embeddings v2
Titan Text G1 - Lite
Titan Text G1 - Lite
Titan Text G1 - Express
Titan Text G1 - Express
Titan Embeddings G1 - Text
Titan Embeddings G1 - Text
Titan Multimodal Embeddings G1
Titan Multimodal Embeddings G1
SDXL 0.8
SDXL 0.8
SDXL 1.0
SDXL 1.0
J2 Grande Instruct
J2 Jumbo Instruct
Jurassic-2 Mid
Jurassic-2 Mid
Jurassic-2 Ultra
Jurassic-2 Ultra
Claude Instant
Claude Instant
Claude
Claude
Claude
Claude
Claude
Claude
Claude
Command
Command
Command Light
Command Light
Embed English
Embed Multilingual
Llama 2 Chat 13B
Llama 2 Chat 13B
Llama 2 Chat 70B
Llama 2 Chat 70B
Llama 2 13B
Llama 2 13B
Llama 2 70B
Llama 2 70B


## Step 2: Test a prompt with Claude

Again, I am relying on `boto3`. The idea here is to test if Claude is accessible and can respond to user queries.

In [4]:
# checking if I can invoke Anthropic Claude
import boto3
import json

bedrock = session.client(service_name='bedrock-runtime', region_name='us-east-1')

modelId = 'anthropic.claude-v2'
accept = 'application/json'
contentType = 'application/json'
body = json.dumps({
    "prompt": "Human:This is a test prompt. Assistant:",
    "max_tokens_to_sample": 300,
    "temperature": 0.1,
    "top_p": 0.9,
})

response = bedrock.invoke_model(body=body, modelId=modelId, accept=accept, contentType=contentType)

response_body = json.loads(response.get('body').read())

print(response_body.get('completion'))


 Hello! I'm Claude, an AI assistant created by Anthropic. I don't actually have personal preferences or experiences to share, since I'm an AI without subjective experiences. I'm designed to be helpful, harmless, and honest through conversations like this.


## Step 3: RAG implementation with Bedrock

For a RAG implementation, I am doing the following:

1. Setting up a runtime model as Anthropic-Claude (`anthropic.claude-v2`)
2. Using a fixed PDF as the document
3. Chroma as my vector database
4. Anthropic-Claude to generate vector embeddings and the AI model

In the [reference blog](https://aws.plainenglish.io/chat-with-your-data-a-simple-guide-using-amazon-bedrock-langchain-and-streamlit-2d60ea857eaf), they mention the use of `DirectoryLoader`. When I tried it, I kept running into library issues with `unstructured`. After fixing it, I realized that Chroma database does not support such data structures. Hence I reverted back to using a single PDF.

In [9]:
import boto3
import json
import time
import os
from dotenv import load_dotenv

from langchain.embeddings import BedrockEmbeddings
from langchain.llms.bedrock import Bedrock
from langchain.text_splitter import RecursiveCharacterTextSplitter
#from langchain.document_loaders import DirectoryLoader
from langchain.document_loaders import PyPDFLoader
from langchain.chains.question_answering import load_qa_chain
from langchain.vectorstores import Chroma

In [30]:
# Setup the boto session and the model to be used. My AWS profile `aws_sol` is stored in the ~/.aws/credentials file.
# When using with Okta, use [Okta-CLI](https://github.com/okta/okta-aws-cli) to generate temporary credentials.

session = boto3.Session(profile_name='aws_sol')
bedrock_runtime = session.client(
    service_name = "bedrock-runtime",
    region_name = "us-east-1"
)

modelId = 'anthropic.claude-v2'
accept = 'application/json'
contentType = 'application/json'
body = json.dumps({
    "max_tokens_to_sample": 40000,
    "temperature": 0.1,
    "top_p": 0.9,
})

In [15]:
# Define the path to the directory containing the PDF files (example_data folder).
file = './data/2212.08073.pdf'

# Function to load documents from the specified directory.
def load_docs(file):
    # Create an instance of the DirectoryLoader with the provided directory path.
    loader = PyPDFLoader(file)
    # Use the loader to load the documents from the directory and store them in 'documents'.
    documents = loader.load()
    # Return the loaded documents.
    return documents


# Call the load_docs function to retrieve the documents from the specified directory.
documents = load_docs(file)

In [16]:
# Function to split the loaded documents into semantically separate chunks.
def split_docs(documents, chunk_size=256, chunk_overlap=25):
    # Create an instance of the RecursiveCharacterTextSplitter with specified chunk size and overlap.
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
    # Use the text splitter to split the documents into chunks and store them in 'docs'.
    docs = text_splitter.split_documents(documents)
    # Return the split documents.
    return docs

# Call the split_docs function to break the loaded documents into chunks.
# The chunk_size and chunk_overlap parameters can be adjusted based on specific requirements.
docs = split_docs(documents)

In [17]:
llm = Bedrock(
    model_id=modelId,
    client=bedrock_runtime
)
bedrock_embeddings = BedrockEmbeddings(client=bedrock_runtime)

In [18]:
# store the data to Chroma DB
db = Chroma.from_documents(documents=docs,embedding=bedrock_embeddings,persist_directory='./data')
db.persist()

In [25]:
# now run a query based on the paper
chain = load_qa_chain(llm, chain_type = "stuff")
query = "What the argument for using a constitutional AI?"
docs = db.similarity_search(query)
resp = chain.run(input_documents = docs, question = query)

In [26]:
print(resp)

 Based on the provided context, the main argument for using a constitutional AI seems to be that it allows the AI system to be developed in a simple, transparent, and controllable way that makes it easier to understand and evaluate its decision-making. Specifically:

- The constitution consists of human-written principles that guide the AI system's development and behavior. This allows human values and preferences to be directly encoded.

- The constitutional approach results in an AI system with a simple and transparent form. This makes it easier to understand how the system makes decisions. 

- Setting constraints via the constitution allows the developers to control aspects of the AI system's behavior (e.g. adopting a certain persona). This makes the system's behavior more predictable and controllable.

- Having an explicit constitution makes it easier to study how different AI behaviors generalize. The developers can directly modify the constitution and evaluate the impacts.

So in

In [27]:
# try a query where the model or the paper may not have correct response
chain = load_qa_chain(llm, chain_type = "stuff")
query = "What the primary issues with RLFHM?"
docs = db.similarity_search(query)
resp = chain.run(input_documents = docs, question = query)

In [28]:
print(resp)

 I do not have enough context to determine the primary issues with RLFHM. The details provided mention it was evaluated on prompts from Thoppilan et al. 2022 and includes responses from HH RLHF models, but do not specify what RLFHM refers to or what issues were found during evaluation. Without more specifics on the model, training approach, and evaluation results, I cannot confidently summarize the primary issues. I would need more context about the evaluation and model details to provide a helpful answer.
